In [24]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import tomllib
from pathlib import Path
from pandas import DataFrame
from common.utils import load_dataset, optimize_memory, get_params, DatasetType
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import joblib
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
import xgboost as xgb
import optuna
import json
import pickle

In [25]:
color = sns.color_palette()

MODEL_VERSION = "1.0.0"
MODEL_PATH = f"../models/ventilator_pressure-prediction_model_{MODEL_VERSION}.pkl"

# Google Brain - Ventilator Pressure Prediction

<br />
 
##### Overview

---
<div><p>What do doctors do when a patient has trouble breathing? They use a ventilator to pump oxygen into a sedated patient's lungs via a tube in the windpipe. But mechanical ventilation is a clinician-intensive procedure, a limitation that was prominently on display during the early days of the COVID-19 pandemic. At the same time, developing new methods for controlling mechanical ventilators is prohibitively expensive, even before reaching clinical trials. High-quality simulators could reduce this barrier. </p>
<p>Current simulators are trained as an ensemble, where each model simulates a single lung setting. However, lungs and their attributes form a continuous space, so a parametric approach must be explored that would consider the differences in patient lungs. </p>
<p>Partnering with Princeton University, the team at Google Brain aims to grow the community around machine learning for mechanical ventilation control. They believe that neural networks and deep learning can better generalize across lungs with varying characteristics than the current industry standard of PID controllers.  </p>
<p>In this competition, you’ll simulate a ventilator connected to a sedated patient's lung. The best submissions will take lung attributes compliance and resistance into account.</p>
<p>If successful, you'll help overcome the cost barrier of developing new methods for controlling mechanical ventilators. This will pave the way for algorithms that adapt to patients and reduce the burden on clinicians during these novel times and beyond. As a result, ventilator treatments may become more widely available to help patients breathe.</p>

##### Evaluation

---

The competition will be scored as the mean absolute error between the predicted and actual pressures during the inspiratory phase of each breath. The expiratory phase is not scored. The score is given by:

|X−Y|

where X
is the vector of predicted pressure and Y

is the vector of actual pressures across all breaths in the test set.
Submission File

For each id in the test set, you must predict a value for the pressure variable. The file should contain a header and have the following format:

<br/>

##### Submission File

----

For each id in the test set, you must predict a value for the pressure variable. The file should contain a header and have the following format:

```csv
id,pressure
1,20
2,23
3,24
etc.
```

##### Dataset Description

---

The ventilator data used in this competition was produced using a modified open-source ventilator connected to an artificial bellows test lung via a respiratory circuit. The diagram below illustrates the setup, with the two control inputs highlighted in green and the state variable (airway pressure) to predict in blue. The first control input is a continuous variable from 0 to 100 representing the percentage the inspiratory solenoid valve is open to let air into the lung (i.e., 0 is completely closed and no air is let in and 100 is completely open). The second control input is a binary variable representing whether the exploratory valve is open (1) or closed (0) to let air out.

In this competition, participants are given numerous time series of breaths and will learn to predict the airway pressure in the respiratory circuit during the breath, given the time series of control inputs.

![image](../docs/ventilator-diagram.png)

Each time series represents an approximately 3-second breath. The files are organized such that each row is a time step in a breath and gives the two control signals, the resulting airway pressure, and relevant attributes of the lung, described below.

<br>

<table>
  <thead>
    <tr>
      <th>File</th>
      <th>Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>train.csv</td>
      <td>The training set</td>
    </tr>
    <tr>
      <td>test.csv</td>
      <td>The test set</td>
    </tr>
    <tr>
      <td>sample_submission.csv</td>
      <td>A sample submission file in the correct format</td>
    </tr>
  </tbody>
</table>

<br>

<table>
  <thead>
    <tr>
      <th>Column</th>
      <th>Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>id</td>
      <td>Globally-unique time step identifier across an entire file</td>
    </tr>
    <tr>
      <td>breath_id</td>
      <td>Globally-unique time step for breaths</td>
    </tr>
    <tr>
      <td>R</td>
      <td>Lung attribute indicating how restricted the airway is (in cmH2O/L/S). This is the change in pressure per change in flow.</td>
    </tr>
    <tr>
      <td>C</td>
      <td>Lung attribute indicating how compliant the lung is (in mL/cmH2O). This is the change in volume per change in pressure.</td>
    </tr>
    <tr>
      <td>time_step</td>
      <td>The actual time stamp</td>
    </tr>
    <tr>
      <td>u_in</td>
      <td>The control input for the inspiratory solenoid valve. Ranges from 0 to 100.</td>
    </tr>
    <tr>
      <td>u_out</td>
      <td>The control input for the exploratory solenoid valve. Either 0 or 1.</td>
    </tr>
    <tr>
      <td>pressure</td>
      <td>The airway pressure measured in the respiratory circuit, measured in cmH2O.</td>
    </tr>
  </tbody>
</table>

In [26]:
def objective(trial: optuna.trial.Trial):
    train_df: DataFrame = load_dataset("ventilator-pressure-prediction", DatasetType.TRAIN,
                                       index=False)

    # train_df, _ = optimize_memory(train_df)

    X_train, X_test, y_train, y_test = train_test_split(
        train_df.drop('pressure', axis=1),
        train_df['pressure'],
        test_size=0.2,
        random_state=42)

    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 100, step=5),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05, step=0.04),
        'max_depth': trial.suggest_int('max_depth', 4, 6, step=2),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.5),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.5),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 100.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 100.0),
        'random_state': 42,
        'verbosity': 3
    }

    model = XGBRegressor(**params)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred)

    return rmse


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

print("Best hyperparameters: ", study.best_params)
print("Best RMSE: ", study.best_value)

[I 2025-04-27 18:49:03,529] A new study created in memory with name: no-name-ec319a30-7a30-4095-a77f-5f300628e853


[18:49:08] ======== Monitor (0): HostSketchContainer ========
[18:49:08] AllReduce: 0.00368s, 1 calls @ 3680us

[18:49:08] MakeCuts: 0.003713s, 1 calls @ 3713us

[18:49:08] DEBUG: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\gbm\gbtree.cc:130: Using tree method: 0
[18:49:15] ======== Monitor (0): Learner ========
[18:49:15] Configure: 0.000273s, 1 calls @ 273us

[18:49:15] EvalOneIter: 0.001308s, 90 calls @ 1308us

[18:49:15] GetGradient: 0.301642s, 90 calls @ 301642us

[18:49:15] PredictRaw: 0.005267s, 90 calls @ 5267us

[18:49:15] UpdateOneIter: 7.35804s, 90 calls @ 7358045us

[18:49:15] ======== Monitor (0): GBTree ========
[18:49:15] BoostNewTrees: 7.01032s, 90 calls @ 7010316us

[18:49:15] CommitModel: 0.000108s, 90 calls @ 108us

[18:49:15] ======== Monitor (0): HistUpdater ========
[18:49:15] BuildHistogram: 1.44093s, 270 calls @ 1440930us

[18:49:15] EvaluateSplits: 0.028935s, 360 calls @ 28935us

[18:49:

[I 2025-04-27 18:49:15,758] Trial 0 finished with value: 31.719075730638 and parameters: {'n_estimators': 90, 'learning_rate': 0.01, 'max_depth': 4, 'subsample': 1.0, 'colsample_bytree': 1.0, 'reg_alpha': 67.79029389500496, 'reg_lambda': 23.371504496380762}. Best is trial 0 with value: 31.719075730638.


[18:49:19] ======== Monitor (0): HostSketchContainer ========
[18:49:19] AllReduce: 0.003418s, 1 calls @ 3418us

[18:49:19] MakeCuts: 0.003449s, 1 calls @ 3449us

[18:49:20] DEBUG: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\gbm\gbtree.cc:130: Using tree method: 0
[18:49:25] ======== Monitor (0): Learner ========
[18:49:25] Configure: 0.000235s, 1 calls @ 235us

[18:49:25] EvalOneIter: 0.000733s, 55 calls @ 733us

[18:49:25] GetGradient: 0.164732s, 55 calls @ 164732us

[18:49:25] PredictRaw: 0.005348s, 55 calls @ 5348us

[18:49:25] UpdateOneIter: 5.81408s, 55 calls @ 5814076us

[18:49:25] ======== Monitor (0): GBTree ========
[18:49:25] BoostNewTrees: 5.60316s, 55 calls @ 5603163us

[18:49:25] CommitModel: 7.5e-05s, 55 calls @ 75us

[18:49:25] ======== Monitor (0): HistUpdater ========
[18:49:25] BuildHistogram: 0.752593s, 165 calls @ 752593us

[18:49:25] EvaluateSplits: 0.007628s, 220 calls @ 7628us

[18:49:25]

[I 2025-04-27 18:49:26,030] Trial 1 finished with value: 43.1934995622756 and parameters: {'n_estimators': 55, 'learning_rate': 0.01, 'max_depth': 4, 'subsample': 0.5, 'colsample_bytree': 0.5, 'reg_alpha': 96.96206669745565, 'reg_lambda': 32.10421397248394}. Best is trial 0 with value: 31.719075730638.


...

[18:50:39] ======== Monitor (0): HostSketchContainer ========
[18:50:39] AllReduce: 0.005346s, 1 calls @ 5346us

[18:50:39] MakeCuts: 0.005428s, 1 calls @ 5428us

[18:50:39] DEBUG: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\gbm\gbtree.cc:130: Using tree method: 0
[18:50:52] ======== Monitor (0): Learner ========
[18:50:52] Configure: 0.000319s, 1 calls @ 319us

[18:50:52] EvalOneIter: 0.001294s, 90 calls @ 1294us

[18:50:52] GetGradient: 0.299736s, 90 calls @ 299736us

[18:50:52] PredictRaw: 0.006849s, 90 calls @ 6849us

[18:50:52] UpdateOneIter: 13.5044s, 90 calls @ 13504427us

[18:50:52] ======== Monitor (0): GBTree ========
[18:50:52] BoostNewTrees: 13.1554s, 90 calls @ 13155431us

[18:50:52] CommitModel: 0.000118s, 90 calls @ 118us

[18:50:52] ======== Monitor (0): HistUpdater ========
[18:50:52] BuildHistogram: 2.93578s, 448 calls @ 2935782us

[18:50:52] EvaluateSplits: 0.038081s, 538 calls @ 38081us

[18:

[I 2025-04-27 18:50:53,509] Trial 8 finished with value: 34.87302925609143 and parameters: {'n_estimators': 90, 'learning_rate': 0.01, 'max_depth': 6, 'subsample': 0.5, 'colsample_bytree': 0.5, 'reg_alpha': 77.41348546813101, 'reg_lambda': 83.34149934108409}. Best is trial 6 with value: 18.817536746748125.


[18:50:58] ======== Monitor (0): HostSketchContainer ========
[18:50:58] AllReduce: 0.008781s, 1 calls @ 8781us

[18:50:58] MakeCuts: 0.008822s, 1 calls @ 8822us

[18:50:58] DEBUG: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\gbm\gbtree.cc:130: Using tree method: 0


[I 2025-04-27 18:51:06,108] Trial 9 finished with value: 40.875705892051805 and parameters: {'n_estimators': 65, 'learning_rate': 0.01, 'max_depth': 4, 'subsample': 0.5, 'colsample_bytree': 0.5, 'reg_alpha': 7.537264077087295, 'reg_lambda': 30.476834109992385}. Best is trial 6 with value: 18.817536746748125.


[18:51:05] ======== Monitor (0): Learner ========
[18:51:05] Configure: 0.000235s, 1 calls @ 235us

[18:51:05] EvalOneIter: 0.000882s, 65 calls @ 882us

[18:51:05] GetGradient: 0.209106s, 65 calls @ 209106us

[18:51:05] PredictRaw: 0.004814s, 65 calls @ 4814us

[18:51:05] UpdateOneIter: 7.41805s, 65 calls @ 7418049us

[18:51:05] ======== Monitor (0): GBTree ========
[18:51:05] BoostNewTrees: 7.1623s, 65 calls @ 7162302us

[18:51:05] CommitModel: 9.6e-05s, 65 calls @ 96us

[18:51:05] ======== Monitor (0): HistUpdater ========
[18:51:05] BuildHistogram: 1.0408s, 195 calls @ 1040797us

[18:51:05] EvaluateSplits: 0.012035s, 260 calls @ 12035us

[18:51:05] InitData: 1.06923s, 65 calls @ 1069225us

[18:51:05] InitRoot: 0.526521s, 65 calls @ 526521us

[18:51:05] LeafPartition: 1.7e-05s, 65 calls @ 17us

[18:51:05] UpdatePosition: 1.9547s, 260 calls @ 1954698us

[18:51:05] UpdatePredictionCache: 0.377684s, 65 calls @ 377684us

[18:51:05] UpdateTree: 4.60425s, 65 calls @ 4604253us

[18:51:05] D

In [28]:
best_params = study.best_trial.params
best_params.update({
    'verbosity': 3,
})

with open(MODEL_PATH, "wb") as f:
    pickle.dump(best_params, f)

In [30]:
model: XGBRegressor = XGBRegressor()

with open(MODEL_PATH, "rb") as f:
    params: dict = pickle.load(f)
model = XGBRegressor(**params)

model.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 1.0,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.05,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 6,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 50,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': 63.435440460708385,
 'reg_lambda': 82.89291381980084,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 1.0,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': 3}